In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/personal_projects/sentiment_analysis_on_noisy_bangla_texts

/content/drive/My Drive/Colab Notebooks/personal_projects/sentiment_analysis_on_noisy_bangla_texts


In [ ]:
#import libraries
import pandas as pd

In [ ]:
#read train, test, validation dataset
train = pd.read_csv('./data/Train.csv')
val   = pd.read_csv('./data/Val.csv')
test  = pd.read_csv('./data/Test.csv')

Data Preprocessing

In [ ]:
!pip install bnlp_toolkit

In [ ]:
from bnlp.corpus import stopwords, punctuations, letters, digits

In [ ]:
from bnlp import BasicTokenizer

In [ ]:
from itertools import chain

In [ ]:
def remove_bengalistopwords(sentence):
    basic_tokenizer = BasicTokenizer()
    tokens = basic_tokenizer.tokenize(sentence)

    bengali_stopwords     = list(stopwords) 
    bengali_punctuations  = list(punctuations)
    bengali_digits        = list(digits)

    remove_list = list(chain(bengali_stopwords, bengali_punctuations, bengali_digits))

    tokens_filtered = [word for word in tokens if not word in remove_list]

    return (" ").join(tokens_filtered)

In [ ]:
train.columns

Index(['Data', 'Label'], dtype='object')

In [ ]:
train['Data'] = train['Data'].apply(remove_bengalistopwords) 

In [ ]:
val['Data'] = val['Data'].apply(remove_bengalistopwords) 

Modeling

In [ ]:
#
X = train.iloc[:,0:1]
y = train['Label'].values

In [ ]:
#
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
# Applying BoW
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(ngram_range=(1,2), binary =True, max_features = 5000)

In [ ]:
X_train_bow = cv.fit_transform(X_train['Data']).toarray()
X_test_bow = cv.transform(X_test['Data']).toarray()

Gaussian NB

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train_bow,y_train)

GaussianNB()

In [ ]:
y_pred = gnb.predict(X_test_bow)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.38767395626242546

In [ ]:
confusion_matrix(y_test,y_pred)

array([[455,  37,  78],
       [640, 248, 162],
       [583,  40, 272]])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred, labels=[1, 2, 0]))

              precision    recall  f1-score   support

           1       0.76      0.24      0.36      1050
           2       0.53      0.30      0.39       895
           0       0.27      0.80      0.40       570

    accuracy                           0.39      2515
   macro avg       0.52      0.45      0.38      2515
weighted avg       0.57      0.39      0.38      2515



Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.5729622266401591

In [ ]:
confusion_matrix(y_test,y_pred)

array([[221, 180, 169],
       [192, 669, 189],
       [170, 174, 551]])

In [ ]:
print(classification_report(y_test, y_pred, labels=[1, 2, 0]))

              precision    recall  f1-score   support

           1       0.65      0.64      0.65      1050
           2       0.61      0.62      0.61       895
           0       0.38      0.39      0.38       570

    accuracy                           0.57      2515
   macro avg       0.55      0.55      0.55      2515
weighted avg       0.57      0.57      0.57      2515

